In [28]:
import pandas as pd
import numpy as np

In [43]:
results_df = pd.read_csv('/Users/kristianertresvag/Documents/Handelshøyskolen BI/Master/4. semester/Master Thesis/Master Thesis/Analysis/results.csv')


## Average Symmetric MAPE (sMAPE): All Data

In [95]:
import pandas as pd
import numpy as np

# Function to calculate sMAPE
def calculate_smape(forecast, actual):
    numerator = np.abs(forecast - actual)
    denominator = (np.abs(actual) + np.abs(forecast)) / 2
    denominator = np.where(denominator == 0, 0.001, denominator)
    smape_elements = 100 * (numerator / denominator)
    smape = smape_elements.sum() / len(smape_elements)  
    return round(smape, 2)

forecast_cols = [col for col in results_df.columns if col not in ['Unnamed: 0', 'Series', 'Date', 'ForecastPeriod', 'Category', 'NF', 'Actual']]

smape_values = pd.DataFrame(columns=['Method', 'ForecastPeriod', 'sMAPE'])

results = []
for method in forecast_cols:
    for period in sorted(results_df['ForecastPeriod'].unique()):
        actual_values = results_df[results_df['ForecastPeriod'] == period]['Actual'].values
        forecast_values = results_df[results_df['ForecastPeriod'] == period][method].values
        smape = calculate_smape(forecast_values, actual_values)
        results.append({'Method': method, 'ForecastPeriod': period, 'sMAPE': smape})

smape_values = pd.DataFrame(results)
smape_pivot_table = smape_values.pivot(index='Method', columns='ForecastPeriod', values='sMAPE')

for col_range in [(1, 4), (1, 6), (1, 8), (1, 12), (1, 15), (1, 18)]:
    cols = [col for col in range(col_range[0], col_range[1] + 1)]
    col_name = f"Avg {col_range[0]}-{col_range[1]}"
    smape_pivot_table[col_name] = round(smape_pivot_table[cols].mean(axis=1), 2)


unique_series_count = {method: results_df.groupby(method)['Series'].nunique() for method in forecast_cols}

smape_pivot_table['#obs'] = np.max(results_df['Series'])

custom_order = [
    'NAIVE2',
    'SINGLE',
    'HOLT',
    'DAMPEN',
    'WINTER',
    'COMB S-H-D',
    'B-J auto',
    'AutoBox1',
    'AutoBox2',
    'AutoBox3',
    'ROBUST-Trend',
    'ARARMA',
    'Auto-ANN',
    'Flors-Pearc1',
    'Flors-Pearc2',
    'PP-Autocast',
    'ForecastPro',
    'SMARTFCS',
    'THETAsm',
    'THETA',
    'RBF',
    'ForcX',
    'Chronos',
    'TimeGPT',
    'Moirai'
]

smape_pivot_table = smape_pivot_table.reindex(custom_order)


smape_pivot_table.to_csv('smape_table.csv')

## Comparison of various methods with Naive2 as the benchmark

In [96]:
var_methods = [
            'THETA',
            'ForecastPro',
            'ForcX',
            'COMB S-H-D',
            'DAMPEN',
            'RBF',
            'TimeGPT',
            'Chronos',
            'Moirai'
]

selected_columns = [1, 'Avg 1-4', 'Avg 1-6', 'Avg 1-12', 'Avg 1-18']

naive2_scores = smape_pivot_table.loc['NAIVE2']

# Calculate differences: sMAPE(NAIVE2) - sMAPE(selected method)
difference_from_naive2 = naive2_scores - smape_pivot_table

filtered_df = difference_from_naive2.loc[var_methods, selected_columns]
sorted_df = filtered_df.sort_values(by=[1, 'Avg 1-4', 'Avg 1-6'], ascending=[False, False, False])

sorted_df.to_csv('smape_comparison.csv')